In [ ]:
    # https://medium.com/@pouryaayria/k-fold-target-encoding-dfe9a594874b
    # https://catboost.ai/

In [ ]:
!pip install catboost

In [ ]:
!pip install --upgrade sklearn

In [ ]:
!pip install seaborn

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import numpy.ma as ma
from sklearn import base
from sklearn.model_selection import KFold
# from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from scipy import stats
import json

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print ("Train data shape:", train.shape)
print ("Test data shape:", test.shape)

Train data shape: (1460, 81)
Test data shape: (1459, 80)


In [4]:
train.head(20
        )

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [5]:
# impute the missing values and create the missing value indicator variables for each numeric column.
train_numeric = train.select_dtypes(include=[np.number])
numeric_cols = train_numeric.columns.values

for col in numeric_cols:
    missing = train[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        train['{}_ismissing'.format(col)] = missing
        med = train[col].median()
        train[col] = train[col].fillna(med)

imputing missing values for: LotFrontage
imputing missing values for: MasVnrArea
imputing missing values for: GarageYrBlt


In [6]:
train.dtypes

Id                         int64
MSSubClass                 int64
MSZoning                  object
LotFrontage              float64
LotArea                    int64
                          ...   
SaleCondition             object
SalePrice                  int64
LotFrontage_ismissing       bool
MasVnrArea_ismissing        bool
GarageYrBlt_ismissing       bool
Length: 84, dtype: object

In [7]:

cols_to_drop = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_less = train.drop(cols_to_drop, axis=1, inplace=True)

In [8]:
# Процентный список пропущенных данных
for col in train.columns:
    pct_missing = np.mean(train[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))
    
    

Id - 0.0%
MSSubClass - 0.0%
MSZoning - 0.0%
LotFrontage - 0.0%
LotArea - 0.0%
Street - 0.0%
LotShape - 0.0%
LandContour - 0.0%
Utilities - 0.0%
LotConfig - 0.0%
LandSlope - 0.0%
Neighborhood - 0.0%
Condition1 - 0.0%
Condition2 - 0.0%
BldgType - 0.0%
HouseStyle - 0.0%
OverallQual - 0.0%
OverallCond - 0.0%
YearBuilt - 0.0%
YearRemodAdd - 0.0%
RoofStyle - 0.0%
RoofMatl - 0.0%
Exterior1st - 0.0%
Exterior2nd - 0.0%
MasVnrType - 1.0%
MasVnrArea - 0.0%
ExterQual - 0.0%
ExterCond - 0.0%
Foundation - 0.0%
BsmtQual - 3.0%
BsmtCond - 3.0%
BsmtExposure - 3.0%
BsmtFinType1 - 3.0%
BsmtFinSF1 - 0.0%
BsmtFinType2 - 3.0%
BsmtFinSF2 - 0.0%
BsmtUnfSF - 0.0%
TotalBsmtSF - 0.0%
Heating - 0.0%
HeatingQC - 0.0%
CentralAir - 0.0%
Electrical - 0.0%
1stFlrSF - 0.0%
2ndFlrSF - 0.0%
LowQualFinSF - 0.0%
GrLivArea - 0.0%
BsmtFullBath - 0.0%
BsmtHalfBath - 0.0%
FullBath - 0.0%
HalfBath - 0.0%
BedroomAbvGr - 0.0%
KitchenAbvGr - 0.0%
KitchenQual - 0.0%
TotRmsAbvGrd - 0.0%
Functional - 0.0%
Fireplaces - 0.0%
GarageType

In [20]:
train = train.dropna()

In [21]:
#One-hot encoding
categorical_columns = []
for col in train.columns:
    if np.sum(train[col].apply(lambda x: type(x) == str)) > 0:
        print(col, 'is categorical', train[col][0:10])
        categorical_columns.append(col)
    else:
        print(col, 'is non categorical', train[col][0:10])

Id is non categorical 0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: Id, dtype: int64
MSSubClass is non categorical 0     60
1     20
2     60
3     70
4     60
5     50
6     20
7     60
8     50
9    190
Name: MSSubClass, dtype: int64
MSZoning is categorical 0    RL
1    RL
2    RL
3    RL
4    RL
5    RL
6    RL
7    RL
8    RM
9    RL
Name: MSZoning, dtype: object
LotFrontage is non categorical 0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
5    85.0
6    75.0
7    69.0
8    51.0
9    50.0
Name: LotFrontage, dtype: float64
LotArea is non categorical 0     8450
1     9600
2    11250
3     9550
4    14260
5    14115
6    10084
7    10382
8     6120
9     7420
Name: LotArea, dtype: int64
Street is categorical 0    Pave
1    Pave
2    Pave
3    Pave
4    Pave
5    Pave
6    Pave
7    Pave
8    Pave
9    Pave
Name: Street, dtype: object
LotShape is categorical 0    Reg
1    Reg
2    IR1
3    IR1
4    IR1
5    IR1
6    Reg
7    IR1
8    Reg
9    

In [22]:
# categorical_columns = train.columns[train.dtypes == 'object']
categorical_columns = pd.Index(categorical_columns)

In [23]:

onehot_encoder = OneHotEncoder(sparse=False)

encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(train[categorical_columns]))
encoded_categorical_columns.head()

,0,1,2,3,4,5,6,7,8,9,...,219,220,221,222,223,224,225,226,227,228
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# train_test_split

In [24]:
train.dtypes

Id                         int64
MSSubClass                 int64
MSZoning                  object
LotFrontage              float64
LotArea                    int64
                          ...   
SaleCondition             object
SalePrice                  int64
LotFrontage_ismissing       bool
MasVnrArea_ismissing        bool
GarageYrBlt_ismissing       bool
Length: 79, dtype: object

In [25]:
X = train.drop(['SalePrice', 'Id'],axis=1)
y = train['SalePrice']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
# Распределение фичей

In [28]:
test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [29]:
# catboost

In [30]:
from catboost import CatBoostRegressor

In [31]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [32]:

model= CatBoostRegressor(iterations=2000,learning_rate=1,depth = 2)

In [33]:
model.fit(X_train,y_train,categorical_features_indices)

0:	learn: 60798.5433771	total: 52.8ms	remaining: 1m 45s
1:	learn: 53290.9985281	total: 57.8ms	remaining: 57.8s
2:	learn: 50900.9615724	total: 63.9ms	remaining: 42.5s
3:	learn: 46486.9313714	total: 66.7ms	remaining: 33.3s
4:	learn: 44507.6279445	total: 70.8ms	remaining: 28.2s
5:	learn: 42519.7272484	total: 74ms	remaining: 24.6s
6:	learn: 41272.3362202	total: 77.8ms	remaining: 22.2s
7:	learn: 39120.0415661	total: 80.9ms	remaining: 20.2s
8:	learn: 37598.4537506	total: 84.4ms	remaining: 18.7s
9:	learn: 37040.1146749	total: 88.3ms	remaining: 17.6s
10:	learn: 36269.9227888	total: 92.3ms	remaining: 16.7s
11:	learn: 35301.3783397	total: 96.2ms	remaining: 15.9s
12:	learn: 35089.6670979	total: 100ms	remaining: 15.3s
13:	learn: 35070.2088746	total: 104ms	remaining: 14.7s
14:	learn: 34757.6958113	total: 107ms	remaining: 14.2s
15:	learn: 34036.3397676	total: 111ms	remaining: 13.8s
16:	learn: 33297.0625637	total: 115ms	remaining: 13.4s
17:	learn: 32976.4524127	total: 118ms	remaining: 13s
18:	learn: 

168:	learn: 20354.5222335	total: 837ms	remaining: 9.07s
169:	learn: 20349.2497505	total: 842ms	remaining: 9.07s
170:	learn: 20325.5922696	total: 847ms	remaining: 9.06s
171:	learn: 20280.6057048	total: 852ms	remaining: 9.05s
172:	learn: 20263.0042454	total: 856ms	remaining: 9.04s
173:	learn: 20203.1551725	total: 861ms	remaining: 9.03s
174:	learn: 20190.1091998	total: 868ms	remaining: 9.05s
175:	learn: 20131.2256826	total: 872ms	remaining: 9.04s
176:	learn: 20082.3613853	total: 880ms	remaining: 9.06s
177:	learn: 20054.3567415	total: 885ms	remaining: 9.06s
178:	learn: 20042.1303644	total: 890ms	remaining: 9.05s
179:	learn: 20008.7382440	total: 894ms	remaining: 9.04s
180:	learn: 19979.6171590	total: 898ms	remaining: 9.03s
181:	learn: 19904.9578062	total: 902ms	remaining: 9.01s
182:	learn: 19904.8776676	total: 906ms	remaining: 8.99s
183:	learn: 19877.6201857	total: 913ms	remaining: 9.01s
184:	learn: 19834.6541725	total: 917ms	remaining: 8.99s
185:	learn: 19830.0607392	total: 920ms	remaining

329:	learn: 16728.1557237	total: 1.62s	remaining: 8.2s
330:	learn: 16672.8009710	total: 1.63s	remaining: 8.2s
331:	learn: 16656.2299554	total: 1.63s	remaining: 8.19s
332:	learn: 16629.0396944	total: 1.63s	remaining: 8.18s
333:	learn: 16596.5844547	total: 1.64s	remaining: 8.18s
334:	learn: 16594.2155220	total: 1.65s	remaining: 8.19s
335:	learn: 16587.9095057	total: 1.66s	remaining: 8.22s
336:	learn: 16577.5586147	total: 1.67s	remaining: 8.24s
337:	learn: 16540.2666484	total: 1.68s	remaining: 8.24s
338:	learn: 16486.8219672	total: 1.68s	remaining: 8.24s
339:	learn: 16481.0667349	total: 1.69s	remaining: 8.24s
340:	learn: 16475.7150002	total: 1.69s	remaining: 8.23s
341:	learn: 16419.0794805	total: 1.7s	remaining: 8.23s
342:	learn: 16376.8043369	total: 1.7s	remaining: 8.23s
343:	learn: 16365.3008488	total: 1.71s	remaining: 8.22s
344:	learn: 16341.6133924	total: 1.71s	remaining: 8.21s
345:	learn: 16320.0935433	total: 1.72s	remaining: 8.22s
346:	learn: 16314.0856194	total: 1.72s	remaining: 8.

496:	learn: 14052.9077507	total: 2.4s	remaining: 7.26s
497:	learn: 14020.4896867	total: 2.41s	remaining: 7.26s
498:	learn: 14015.5905948	total: 2.41s	remaining: 7.25s
499:	learn: 14010.4992641	total: 2.42s	remaining: 7.25s
500:	learn: 13998.1572686	total: 2.42s	remaining: 7.25s
501:	learn: 13989.8231483	total: 2.43s	remaining: 7.25s
502:	learn: 13980.1117025	total: 2.43s	remaining: 7.24s
503:	learn: 13979.4465446	total: 2.44s	remaining: 7.24s
504:	learn: 13940.3858273	total: 2.44s	remaining: 7.23s
505:	learn: 13928.2137948	total: 2.45s	remaining: 7.23s
506:	learn: 13896.2407592	total: 2.45s	remaining: 7.22s
507:	learn: 13875.8324390	total: 2.46s	remaining: 7.23s
508:	learn: 13875.2304442	total: 2.46s	remaining: 7.22s
509:	learn: 13851.0388278	total: 2.47s	remaining: 7.22s
510:	learn: 13802.6774942	total: 2.48s	remaining: 7.22s
511:	learn: 13781.4752117	total: 2.48s	remaining: 7.21s
512:	learn: 13781.4551940	total: 2.48s	remaining: 7.21s
513:	learn: 13766.4801690	total: 2.49s	remaining:

672:	learn: 12199.7243896	total: 3.19s	remaining: 6.29s
673:	learn: 12174.3310695	total: 3.19s	remaining: 6.28s
674:	learn: 12141.1929942	total: 3.2s	remaining: 6.28s
675:	learn: 12125.8630190	total: 3.2s	remaining: 6.27s
676:	learn: 12120.9172034	total: 3.21s	remaining: 6.27s
677:	learn: 12120.3112965	total: 3.21s	remaining: 6.27s
678:	learn: 12107.0287792	total: 3.22s	remaining: 6.27s
679:	learn: 12092.9243697	total: 3.23s	remaining: 6.26s
680:	learn: 12077.4788667	total: 3.23s	remaining: 6.25s
681:	learn: 12059.3980449	total: 3.23s	remaining: 6.25s
682:	learn: 12056.0417420	total: 3.24s	remaining: 6.24s
683:	learn: 12027.1354764	total: 3.24s	remaining: 6.24s
684:	learn: 12020.1663267	total: 3.25s	remaining: 6.23s
685:	learn: 12009.6413246	total: 3.25s	remaining: 6.23s
686:	learn: 12002.6593812	total: 3.25s	remaining: 6.22s
687:	learn: 11996.3433234	total: 3.26s	remaining: 6.21s
688:	learn: 11974.4136871	total: 3.26s	remaining: 6.21s
689:	learn: 11971.4605362	total: 3.26s	remaining: 

...

1220:	learn: 7680.4130859	total: 5.54s	remaining: 3.53s
1221:	learn: 7675.5932580	total: 5.54s	remaining: 3.53s
1222:	learn: 7675.3886067	total: 5.55s	remaining: 3.52s
1223:	learn: 7670.7698532	total: 5.55s	remaining: 3.52s
1224:	learn: 7670.7667817	total: 5.56s	remaining: 3.52s
1225:	learn: 7668.0898668	total: 5.57s	remaining: 3.51s
1226:	learn: 7663.2287067	total: 5.57s	remaining: 3.51s
1227:	learn: 7654.8092484	total: 5.58s	remaining: 3.51s
1228:	learn: 7649.3375039	total: 5.58s	remaining: 3.5s
1229:	learn: 7649.2807233	total: 5.59s	remaining: 3.5s
1230:	learn: 7644.7209977	total: 5.6s	remaining: 3.5s
1231:	learn: 7632.7371973	total: 5.6s	remaining: 3.49s
1232:	learn: 7627.8042679	total: 5.61s	remaining: 3.49s
1233:	learn: 7625.4774733	total: 5.61s	remaining: 3.48s
1234:	learn: 7621.2004145	total: 5.62s	remaining: 3.48s
1235:	learn: 7621.1627665	total: 5.62s	remaining: 3.48s
1236:	learn: 7619.2919335	total: 5.63s	remaining: 3.47s
1237:	learn: 7618.9010824	total: 5.63s	remaining: 3.4

1381:	learn: 7017.6957455	total: 6.33s	remaining: 2.83s
1382:	learn: 7013.8536265	total: 6.33s	remaining: 2.82s
1383:	learn: 7012.7705244	total: 6.33s	remaining: 2.82s
1384:	learn: 7002.3990233	total: 6.34s	remaining: 2.81s
1385:	learn: 6995.0441555	total: 6.34s	remaining: 2.81s
1386:	learn: 6988.8572652	total: 6.35s	remaining: 2.81s
1387:	learn: 6975.8173617	total: 6.36s	remaining: 2.8s
1388:	learn: 6968.3873808	total: 6.36s	remaining: 2.8s
1389:	learn: 6957.5703966	total: 6.37s	remaining: 2.79s
1390:	learn: 6956.7724838	total: 6.38s	remaining: 2.79s
1391:	learn: 6955.9022973	total: 6.38s	remaining: 2.79s
1392:	learn: 6951.1401976	total: 6.38s	remaining: 2.78s
1393:	learn: 6937.7335690	total: 6.39s	remaining: 2.78s
1394:	learn: 6936.3190020	total: 6.39s	remaining: 2.77s
1395:	learn: 6935.7218013	total: 6.4s	remaining: 2.77s
1396:	learn: 6935.2060042	total: 6.4s	remaining: 2.76s
1397:	learn: 6926.4200053	total: 6.4s	remaining: 2.76s
1398:	learn: 6919.5620976	total: 6.41s	remaining: 2.7

1559:	learn: 6271.8207673	total: 7.11s	remaining: 2s
1560:	learn: 6270.9896826	total: 7.12s	remaining: 2s
1561:	learn: 6264.5402606	total: 7.12s	remaining: 2s
1562:	learn: 6243.9644982	total: 7.13s	remaining: 1.99s
1563:	learn: 6228.4159640	total: 7.13s	remaining: 1.99s
1564:	learn: 6219.5721833	total: 7.14s	remaining: 1.98s
1565:	learn: 6206.7633903	total: 7.14s	remaining: 1.98s
1566:	learn: 6203.8719388	total: 7.15s	remaining: 1.98s
1567:	learn: 6203.2154937	total: 7.15s	remaining: 1.97s
1568:	learn: 6199.9632098	total: 7.16s	remaining: 1.97s
1569:	learn: 6198.9796453	total: 7.17s	remaining: 1.96s
1570:	learn: 6196.7813072	total: 7.17s	remaining: 1.96s
1571:	learn: 6195.3592388	total: 7.18s	remaining: 1.95s
1572:	learn: 6193.4339352	total: 7.18s	remaining: 1.95s
1573:	learn: 6193.2201889	total: 7.19s	remaining: 1.95s
1574:	learn: 6186.3041620	total: 7.19s	remaining: 1.94s
1575:	learn: 6181.9887997	total: 7.2s	remaining: 1.94s
1576:	learn: 6181.5557575	total: 7.2s	remaining: 1.93s
157

1736:	learn: 5531.2541715	total: 7.9s	remaining: 1.2s
1737:	learn: 5530.5775780	total: 7.9s	remaining: 1.19s
1738:	learn: 5525.1625949	total: 7.91s	remaining: 1.19s
1739:	learn: 5524.2939171	total: 7.91s	remaining: 1.18s
1740:	learn: 5522.1312099	total: 7.91s	remaining: 1.18s
1741:	learn: 5522.0487086	total: 7.92s	remaining: 1.17s
1742:	learn: 5493.3210266	total: 7.92s	remaining: 1.17s
1743:	learn: 5492.8975793	total: 7.93s	remaining: 1.16s
1744:	learn: 5487.1565537	total: 7.93s	remaining: 1.16s
1745:	learn: 5486.5673124	total: 7.94s	remaining: 1.15s
1746:	learn: 5485.2583468	total: 7.94s	remaining: 1.15s
1747:	learn: 5483.6959495	total: 7.95s	remaining: 1.15s
1748:	learn: 5483.5979677	total: 7.95s	remaining: 1.14s
1749:	learn: 5482.7832502	total: 7.95s	remaining: 1.14s
1750:	learn: 5481.9219396	total: 7.96s	remaining: 1.13s
1751:	learn: 5476.4185284	total: 7.96s	remaining: 1.13s
1752:	learn: 5475.3128918	total: 7.96s	remaining: 1.12s
1753:	learn: 5472.5101149	total: 7.97s	remaining: 1

1922:	learn: 4991.6304979	total: 8.68s	remaining: 348ms
1923:	learn: 4991.5648171	total: 8.69s	remaining: 343ms
1924:	learn: 4987.6832159	total: 8.69s	remaining: 339ms
1925:	learn: 4985.7856566	total: 8.7s	remaining: 334ms
1926:	learn: 4983.5407009	total: 8.7s	remaining: 330ms
1927:	learn: 4982.9028600	total: 8.71s	remaining: 325ms
1928:	learn: 4976.0670957	total: 8.71s	remaining: 321ms
1929:	learn: 4975.8311406	total: 8.72s	remaining: 316ms
1930:	learn: 4974.6067398	total: 8.72s	remaining: 312ms
1931:	learn: 4972.5840476	total: 8.73s	remaining: 307ms
1932:	learn: 4972.4840841	total: 8.73s	remaining: 303ms
1933:	learn: 4969.1310151	total: 8.74s	remaining: 298ms
1934:	learn: 4966.9958818	total: 8.74s	remaining: 294ms
1935:	learn: 4965.3884142	total: 8.75s	remaining: 289ms
1936:	learn: 4964.3589555	total: 8.75s	remaining: 285ms
1937:	learn: 4964.0248012	total: 8.76s	remaining: 280ms
1938:	learn: 4963.7048567	total: 8.76s	remaining: 276ms
1939:	learn: 4962.3720329	total: 8.77s	remaining: 

In [34]:
# MAE metrics

In [35]:
from sklearn.metrics import mean_absolute_error as mae

In [72]:
 predictions = model.predict(X_test)
 MAE_test_ohe = mae(predictions, y_test)

In [73]:
print(str(MAE_test_ohe))
    

27640.063893391678


In [38]:
X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_ismissing,MasVnrArea_ismissing,GarageYrBlt_ismissing
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,2,2008,WD,Normal,False,False,False
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,5,2007,WD,Normal,False,False,False
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,9,2008,WD,Normal,False,False,False
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,2,2006,WD,Abnorml,False,False,False
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,12,2008,WD,Normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,8,2007,WD,Normal,False,False,False
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,2,2010,WD,Normal,False,False,False
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,2500,5,2010,WD,Normal,False,False,False
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,4,2010,WD,Normal,False,False,False


In [39]:
class KFoldTargetEncoderTrain(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, colnames,targetName,n_fold=5,verbosity=True,discardOriginal_col=False):

        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col

    def fit(self, X, y=None):
        return self


    def transform(self,X):

        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)

        mean_of_target = X[self.targetName].mean()
        kf = KFold(n_splits = self.n_fold, shuffle = False, random_state=2019)



        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan

        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
#             print(tr_ind,val_ind)
            X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].map(X_tr.groupby(self.colnames)[self.targetName].mean())
        X[col_mean_name][X[col_mean_name].apply(lambda x: type(x) == str)] = None
        X[col_mean_name].fillna(mean_of_target, inplace = True)

        if self.verbosity:

            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,
                                                                                      self.targetName,
                                                                                      np.corrcoef(X[self.targetName].values, encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.colnames, axis=1)
            

        return X


In [40]:
categorical_columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [ ]:
# cols_to_drop2 = ['HouseStyle_Kfold_Target_Enc','RoofStyle_Kfold_Target_Enc','RoofMatl_Kfold_Target_Enc','Exterior2nd_Kfold_Target_Enc']
# categorical_columns = X_te_test.drop(cols_to_drop2, axis=1, inplace=True) 

In [ ]:
# categorical_columns.append(pd.Index(["Exterior1st_Kfold_Target_Enc","BsmtQual_Kfold_Target_Enc","BsmtCond_Kfold_Target_Enc","BsmtExposure_Kfold_Target_Enc","BsmtFinType1_Kfold_Target_Enc","BsmtFinType2_Kfold_Target_Enc","Heating_Kfold_Target_Enc"]))

In [41]:
Xy_train = pd.concat([X_train,y_train], 1)
Xy_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_ismissing,MasVnrArea_ismissing,GarageYrBlt_ismissing,SalePrice
1145,50,RM,52.0,6240,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,8,2006,WD,Family,False,False,False,149000
745,60,RL,69.0,8963,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,7,2008,WD,Normal,True,False,False,299800
1134,60,RL,57.0,8773,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0,0,8,2007,WD,Normal,False,False,False,169000
537,20,RL,69.0,12735,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,4,2008,COD,Normal,True,False,False,111250
41,20,RL,115.0,16905,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,7,2007,WD,Normal,False,False,False,170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,60,RL,58.0,14054,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,11,2006,New,Partial,False,False,False,219210
1236,160,RL,36.0,2628,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,6,2010,WD,Normal,False,False,False,175500
1413,20,RL,88.0,10994,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,9,2009,COD,Abnorml,False,False,False,257000
933,20,RL,63.0,8487,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,8,2009,WD,Normal,False,False,False,190000


In [42]:
list(Xy_train.columns)

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',

In [43]:
Xy_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_ismissing,MasVnrArea_ismissing,GarageYrBlt_ismissing,SalePrice
1145,50,RM,52.0,6240,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,8,2006,WD,Family,False,False,False,149000
745,60,RL,69.0,8963,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,7,2008,WD,Normal,True,False,False,299800
1134,60,RL,57.0,8773,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0,0,8,2007,WD,Normal,False,False,False,169000
537,20,RL,69.0,12735,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,4,2008,COD,Normal,True,False,False,111250
41,20,RL,115.0,16905,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,7,2007,WD,Normal,False,False,False,170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,60,RL,58.0,14054,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,11,2006,New,Partial,False,False,False,219210
1236,160,RL,36.0,2628,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,6,2010,WD,Normal,False,False,False,175500
1413,20,RL,88.0,10994,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,9,2009,COD,Abnorml,False,False,False,257000
933,20,RL,63.0,8487,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,8,2009,WD,Normal,False,False,False,190000


In [44]:
for col in categorical_columns:
    Xy_train = KFoldTargetEncoderTrain(colnames=col, targetName="SalePrice", discardOriginal_col=False).transform(Xy_train)

/home/g/miniconda3/envs/bml/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/g/miniconda3/envs/bml/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Correlation between the new feature, MSZoning_Kfold_Target_Enc and, SalePrice is 0.2989737894613584.
Correlation between the new feature, Street_Kfold_Target_Enc and, SalePrice is 0.04467542258125285.
Correlation between the new feature, LotShape_Kfold_Target_Enc and, SalePrice is 0.26088057549311905.
Correlation between the new feature, LandContour_Kfold_Target_Enc and, SalePrice is 0.09950208170866175.
Correlation between the new feature, Utilities_Kfold_Target_Enc and, SalePrice is -0.07838505956187573.
Correlation between the new feature, LotConfig_Kfold_Target_Enc and, SalePrice is 0.10000446121706566.
Correlation between the new feature, LandSlope_Kfold_Target_Enc and, SalePrice is 0.011401489409827532.
Correlation between the new feature, Neighborhood_Kfold_Target_Enc and, SalePrice is 0.704382803297054.
Correlation between the new feature, Condition1_Kfold_Target_Enc and, SalePrice is 0.13110944231043653.
Correlation between the new feature, Condition2_Kfold_Target_Enc and, Sal

In [54]:
class KFoldTargetEncoderTest(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,encodedName):
        
        self.train = train
        self.colNames = colNames
        self.encodedName = encodedName
        
        
    def fit(self, X, y=None):
        return self

    def transform(self,X):


        mean = self.train[[self.colNames,self.encodedName]].groupby(self.colNames).mean().reset_index() 
        print(mean)
        dd = {}
        for index, row in mean.iterrows():
            dd[row[self.colNames]] = row[self.encodedName]

        print(dd)
        X[self.encodedName] = X[self.colNames]
        X = X.replace({self.encodedName: dd})
        
#         for col in X.columns:
#             X[col][X[col].apply(lambda x: type(x) == str)] = None
#             X[col].fillna(self.train[self.encodedName].mean(), inplace = True)
        return X
        

In [55]:
X_te_test = X_test.copy()

In [56]:
for col in categorical_columns:
    X_te_test = KFoldTargetEncoderTest(train=Xy_train, colNames=col, encodedName=col + '_' + 'Kfold_Target_Enc').transform(X_te_test)

  MSZoning  MSZoning_Kfold_Target_Enc
0  C (all)               73755.600000
1       FV              213909.366910
2       RH              136659.047619
3       RL              199257.829536
4       RM              133925.761437
{'C (all)': 73755.6, 'FV': 213909.36691017315, 'RH': 136659.04761904763, 'RL': 199257.82953621598, 'RM': 133925.76143744343}
  Street  Street_Kfold_Target_Enc
0   Grvl             68496.500000
1   Pave            189587.028658
{'Grvl': 68496.5, 'Pave': 189587.02865767805}
  LotShape  LotShape_Kfold_Target_Enc
0      IR1              214668.830627
1      IR2              242944.330220
2      IR3              223591.916667
3      Reg              171217.678569
{'IR1': 214668.83062691867, 'IR2': 242944.33021978018, 'IR3': 223591.91666666666, 'Reg': 171217.67856878694}
  LandContour  LandContour_Kfold_Target_Enc
0         Bnk                 146627.261413
1         HLS                 229085.681534
2         Low                 215988.097744
3         Lvl           

  Foundation  Foundation_Kfold_Target_Enc
0     BrkTil                137998.629358
1     CBlock                155153.980603
2      PConc                229652.645920
3      Stone                244569.333333
4       Wood                174833.333333
{'BrkTil': 137998.62935783042, 'CBlock': 155153.9806025654, 'PConc': 229652.64592032746, 'Stone': 244569.33333333334, 'Wood': 174833.33333333334}
  BsmtQual  BsmtQual_Kfold_Target_Enc
0       Ex              344241.154076
1       Fa              126949.639899
2       Gd              205775.407890
3       TA              145171.713818
{'Ex': 344241.15407647414, 'Fa': 126949.63989898987, 'Gd': 205775.4078897051, 'TA': 145171.71381788302}
  BsmtCond  BsmtCond_Kfold_Target_Enc
0       Fa              135118.166450
1       Gd              214400.671692
2       Po              189316.539062
3       TA              189784.902245
{'Fa': 135118.16645021646, 'Gd': 214400.6716916447, 'Po': 189316.5390625, 'TA': 189784.90224541302}
  BsmtExposure  Bs

In [58]:
for col in categorical_columns:
    Xy_train = Xy_train.drop(col, 1)

In [57]:
for col in categorical_columns:
    X_te_test = X_te_test.drop(col, 1)

In [59]:
model_te = CatBoostRegressor(iterations=2000,learning_rate=1,depth = 2)

In [60]:
model_te.fit(Xy_train.drop('SalePrice', axis=1),Xy_train['SalePrice'])

0:	learn: 58416.8905014	total: 660us	remaining: 1.32s
1:	learn: 49104.3758592	total: 1.46ms	remaining: 1.46s
2:	learn: 45905.6007129	total: 2.04ms	remaining: 1.36s
3:	learn: 40503.6601548	total: 3.05ms	remaining: 1.52s
4:	learn: 37623.6228041	total: 3.94ms	remaining: 1.57s
5:	learn: 35865.7039728	total: 4.8ms	remaining: 1.59s
6:	learn: 33847.0366093	total: 5.63ms	remaining: 1.6s
7:	learn: 32483.9019184	total: 6.57ms	remaining: 1.64s
8:	learn: 30829.8296018	total: 7.24ms	remaining: 1.6s
9:	learn: 29415.6607275	total: 8.03ms	remaining: 1.6s
10:	learn: 27975.9820019	total: 9.01ms	remaining: 1.63s
11:	learn: 26750.1436307	total: 10ms	remaining: 1.66s
12:	learn: 25933.8771875	total: 11.4ms	remaining: 1.74s
13:	learn: 25276.2209603	total: 12.3ms	remaining: 1.74s
14:	learn: 24814.8840007	total: 12.9ms	remaining: 1.71s
15:	learn: 24148.9022537	total: 13.6ms	remaining: 1.68s
16:	learn: 23736.2318493	total: 15.2ms	remaining: 1.77s
17:	learn: 23130.3251115	total: 15.8ms	remaining: 1.74s
18:	learn

285:	learn: 5130.8352705	total: 179ms	remaining: 1.07s
286:	learn: 5104.5008851	total: 182ms	remaining: 1.08s
287:	learn: 5080.1314613	total: 182ms	remaining: 1.08s
288:	learn: 5050.6868440	total: 183ms	remaining: 1.08s
289:	learn: 5026.6242291	total: 184ms	remaining: 1.08s
290:	learn: 4995.3317663	total: 184ms	remaining: 1.08s
291:	learn: 4977.5787198	total: 185ms	remaining: 1.08s
292:	learn: 4951.1439347	total: 185ms	remaining: 1.08s
293:	learn: 4944.0627533	total: 186ms	remaining: 1.08s
294:	learn: 4934.6361272	total: 187ms	remaining: 1.08s
295:	learn: 4918.0157537	total: 187ms	remaining: 1.08s
296:	learn: 4901.5431420	total: 189ms	remaining: 1.08s
297:	learn: 4890.0959275	total: 189ms	remaining: 1.08s
298:	learn: 4883.1940964	total: 190ms	remaining: 1.08s
299:	learn: 4856.6420781	total: 191ms	remaining: 1.08s
300:	learn: 4852.1052491	total: 192ms	remaining: 1.08s
301:	learn: 4835.3419078	total: 192ms	remaining: 1.08s
302:	learn: 4806.7004260	total: 193ms	remaining: 1.08s
303:	learn

561:	learn: 2238.8290628	total: 362ms	remaining: 925ms
562:	learn: 2232.6696615	total: 363ms	remaining: 926ms
563:	learn: 2222.1219352	total: 364ms	remaining: 926ms
564:	learn: 2212.9760777	total: 364ms	remaining: 926ms
565:	learn: 2204.6655561	total: 365ms	remaining: 925ms
566:	learn: 2196.3541678	total: 366ms	remaining: 924ms
567:	learn: 2190.9966290	total: 366ms	remaining: 923ms
568:	learn: 2179.7194152	total: 367ms	remaining: 922ms
569:	learn: 2177.9691428	total: 367ms	remaining: 921ms
570:	learn: 2167.8195769	total: 368ms	remaining: 920ms
571:	learn: 2162.7930448	total: 368ms	remaining: 919ms
572:	learn: 2155.2074673	total: 369ms	remaining: 919ms
573:	learn: 2149.7750109	total: 369ms	remaining: 918ms
574:	learn: 2143.5652727	total: 370ms	remaining: 917ms
575:	learn: 2138.7637063	total: 370ms	remaining: 915ms
576:	learn: 2137.1640296	total: 371ms	remaining: 914ms
577:	learn: 2130.8319758	total: 371ms	remaining: 913ms
578:	learn: 2121.9161479	total: 372ms	remaining: 912ms
579:	learn

776:	learn: 1256.8485169	total: 547ms	remaining: 860ms
777:	learn: 1248.7775623	total: 548ms	remaining: 860ms
778:	learn: 1240.6873566	total: 549ms	remaining: 860ms
779:	learn: 1239.3603194	total: 550ms	remaining: 860ms
780:	learn: 1237.7555995	total: 550ms	remaining: 859ms
781:	learn: 1236.2791678	total: 551ms	remaining: 859ms
782:	learn: 1230.5205557	total: 552ms	remaining: 858ms
783:	learn: 1219.4403350	total: 553ms	remaining: 857ms
784:	learn: 1217.1459937	total: 553ms	remaining: 857ms
785:	learn: 1215.9368428	total: 554ms	remaining: 856ms
786:	learn: 1213.7006873	total: 554ms	remaining: 855ms
787:	learn: 1212.1310250	total: 555ms	remaining: 854ms
788:	learn: 1207.5369179	total: 556ms	remaining: 854ms
789:	learn: 1203.1363822	total: 557ms	remaining: 853ms
790:	learn: 1202.0615400	total: 557ms	remaining: 852ms
791:	learn: 1198.0538231	total: 558ms	remaining: 851ms
792:	learn: 1194.3518304	total: 559ms	remaining: 850ms
793:	learn: 1191.5681550	total: 559ms	remaining: 850ms
794:	learn

1049:	learn: 620.4943319	total: 725ms	remaining: 656ms
1050:	learn: 619.4280512	total: 725ms	remaining: 655ms
1051:	learn: 617.6653778	total: 726ms	remaining: 654ms
1052:	learn: 615.4848396	total: 727ms	remaining: 654ms
1053:	learn: 612.3594805	total: 728ms	remaining: 653ms
1054:	learn: 610.6612753	total: 729ms	remaining: 653ms
1055:	learn: 610.1903550	total: 730ms	remaining: 652ms
1056:	learn: 608.4896353	total: 730ms	remaining: 652ms
1057:	learn: 606.3144465	total: 732ms	remaining: 651ms
1058:	learn: 604.2673006	total: 733ms	remaining: 651ms
1059:	learn: 604.0910284	total: 734ms	remaining: 650ms
1060:	learn: 603.0037912	total: 734ms	remaining: 650ms
1061:	learn: 600.6853486	total: 735ms	remaining: 649ms
1062:	learn: 598.9508463	total: 736ms	remaining: 648ms
1063:	learn: 597.2277983	total: 737ms	remaining: 648ms
1064:	learn: 594.4872411	total: 737ms	remaining: 647ms
1065:	learn: 593.0155855	total: 738ms	remaining: 647ms
1066:	learn: 591.7190954	total: 739ms	remaining: 646ms
1067:	lear

1278:	learn: 357.3631444	total: 907ms	remaining: 511ms
1279:	learn: 356.2392174	total: 908ms	remaining: 511ms
1280:	learn: 355.5728043	total: 909ms	remaining: 510ms
1281:	learn: 354.6384379	total: 909ms	remaining: 509ms
1282:	learn: 353.6541987	total: 910ms	remaining: 509ms
1283:	learn: 352.6849561	total: 911ms	remaining: 508ms
1284:	learn: 352.3968965	total: 911ms	remaining: 507ms
1285:	learn: 351.5416177	total: 914ms	remaining: 507ms
1286:	learn: 350.5509980	total: 914ms	remaining: 507ms
1287:	learn: 350.2551697	total: 915ms	remaining: 506ms
1288:	learn: 349.9657998	total: 916ms	remaining: 505ms
1289:	learn: 349.2114396	total: 917ms	remaining: 505ms
1290:	learn: 347.4737921	total: 918ms	remaining: 504ms
1291:	learn: 346.8392819	total: 920ms	remaining: 504ms
1292:	learn: 346.3692405	total: 922ms	remaining: 504ms
1293:	learn: 346.1772584	total: 923ms	remaining: 504ms
1294:	learn: 344.8985053	total: 924ms	remaining: 503ms
1295:	learn: 344.6910940	total: 925ms	remaining: 502ms
1296:	lear

1481:	learn: 219.8991440	total: 1.09s	remaining: 380ms
1482:	learn: 219.2432083	total: 1.09s	remaining: 380ms
1483:	learn: 219.1650107	total: 1.09s	remaining: 379ms
1484:	learn: 218.6019271	total: 1.09s	remaining: 378ms
1485:	learn: 218.1753834	total: 1.09s	remaining: 377ms
1486:	learn: 218.0659795	total: 1.09s	remaining: 377ms
1487:	learn: 217.3257711	total: 1.09s	remaining: 376ms
1488:	learn: 216.9436985	total: 1.09s	remaining: 375ms
1489:	learn: 216.8199145	total: 1.09s	remaining: 375ms
1490:	learn: 216.5065960	total: 1.09s	remaining: 374ms
1491:	learn: 215.4727029	total: 1.1s	remaining: 373ms
1492:	learn: 214.8839771	total: 1.1s	remaining: 373ms
1493:	learn: 213.6939800	total: 1.1s	remaining: 372ms
1494:	learn: 213.0663277	total: 1.1s	remaining: 371ms
1495:	learn: 212.3144909	total: 1.1s	remaining: 370ms
1496:	learn: 212.1864868	total: 1.1s	remaining: 370ms
1497:	learn: 211.5906385	total: 1.1s	remaining: 369ms
1498:	learn: 211.0142748	total: 1.1s	remaining: 369ms
1499:	learn: 210.7

1713:	learn: 133.4900377	total: 1.27s	remaining: 212ms
1714:	learn: 132.9159156	total: 1.27s	remaining: 211ms
1715:	learn: 132.8489117	total: 1.27s	remaining: 210ms
1716:	learn: 132.7425014	total: 1.27s	remaining: 210ms
1717:	learn: 132.3552120	total: 1.27s	remaining: 209ms
1718:	learn: 131.9351402	total: 1.27s	remaining: 208ms
1719:	learn: 131.8587976	total: 1.27s	remaining: 208ms
1720:	learn: 131.5913751	total: 1.28s	remaining: 207ms
1721:	learn: 131.3238429	total: 1.28s	remaining: 206ms
1722:	learn: 130.7247913	total: 1.28s	remaining: 205ms
1723:	learn: 130.3562800	total: 1.28s	remaining: 205ms
1724:	learn: 129.9446026	total: 1.28s	remaining: 204ms
1725:	learn: 129.3744982	total: 1.28s	remaining: 203ms
1726:	learn: 128.8689345	total: 1.28s	remaining: 202ms
1727:	learn: 128.4787898	total: 1.28s	remaining: 202ms
1728:	learn: 127.7545339	total: 1.28s	remaining: 201ms
1729:	learn: 127.5540359	total: 1.28s	remaining: 200ms
1730:	learn: 127.2504528	total: 1.28s	remaining: 199ms
1731:	lear

1931:	learn: 80.9059328	total: 1.45s	remaining: 51.1ms
1932:	learn: 80.8182904	total: 1.45s	remaining: 50.3ms
1933:	learn: 80.5927506	total: 1.45s	remaining: 49.6ms
1934:	learn: 80.2721334	total: 1.45s	remaining: 48.8ms
1935:	learn: 80.1537521	total: 1.45s	remaining: 48.1ms
1936:	learn: 79.9690650	total: 1.46s	remaining: 47.3ms
1937:	learn: 79.4423523	total: 1.46s	remaining: 46.6ms
1938:	learn: 79.3306756	total: 1.46s	remaining: 45.8ms
1939:	learn: 79.2354815	total: 1.46s	remaining: 45.1ms
1940:	learn: 79.1455259	total: 1.46s	remaining: 44.3ms
1941:	learn: 79.0598078	total: 1.46s	remaining: 43.6ms
1942:	learn: 78.8319215	total: 1.46s	remaining: 42.8ms
1943:	learn: 78.7671040	total: 1.46s	remaining: 42ms
1944:	learn: 78.5715087	total: 1.46s	remaining: 41.3ms
1945:	learn: 78.3498068	total: 1.46s	remaining: 40.5ms
1946:	learn: 77.9672092	total: 1.46s	remaining: 39.8ms
1947:	learn: 77.9392933	total: 1.46s	remaining: 39ms
1948:	learn: 77.8311638	total: 1.46s	remaining: 38.3ms
1949:	learn: 7

In [62]:
X_te_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Electrical_Kfold_Target_Enc,KitchenQual_Kfold_Target_Enc,Functional_Kfold_Target_Enc,GarageType_Kfold_Target_Enc,GarageFinish_Kfold_Target_Enc,GarageQual_Kfold_Target_Enc,GarageCond_Kfold_Target_Enc,PavedDrive_Kfold_Target_Enc,SaleType_Kfold_Target_Enc,SaleCondition_Kfold_Target_Enc
832,60,44.0,9548,7,6,2003,2003,223.0,483,0,...,193797.116943,217322.933366,191444.369385,206448.583496,204640.669623,191439.259129,191530,192389.396129,180432.288762,182035
965,60,65.0,10237,6,5,2005,2007,0.0,0,0,...,193797.116943,217322.933366,191444.369385,206448.583496,247105.474727,191439.259129,191530,192389.396129,284120.716217,281738
969,190,75.0,10382,6,5,1958,1958,105.0,513,0,...,193797.116943,143717.123918,191444.369385,206448.583496,204640.669623,191439.259129,191530,192389.396129,196918.666667,182035
1411,50,80.0,9600,6,8,1950,2005,0.0,120,0,...,193797.116943,143717.123918,157096.257631,206448.583496,142916.051642,191439.259129,191530,192389.396129,180432.288762,182035
284,120,50.0,8012,6,5,1992,1992,0.0,430,0,...,193797.116943,217322.933366,191444.369385,206448.583496,204640.669623,191439.259129,191530,192389.396129,180432.288762,182035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,190,76.0,7630,5,9,1900,1996,0.0,0,0,...,193797.116943,217322.933366,191444.369385,133517.516248,142916.051642,191439.259129,191530,134068.511401,180432.288762,182035
661,60,52.0,46589,8,7,1994,2005,528.0,1361,180,...,193797.116943,217322.933366,191444.369385,206448.583496,204640.669623,191439.259129,191530,192389.396129,180432.288762,182035
798,60,104.0,13518,9,5,2008,2009,860.0,0,0,...,193797.116943,346079.838916,191444.369385,267328.503955,247105.474727,191439.259129,191530,192389.396129,284120.716217,281738
351,120,69.0,5271,7,5,1986,1986,0.0,1082,0,...,193797.116943,217322.933366,191444.369385,206448.583496,204640.669623,191439.259129,191530,192389.396129,180432.288762,160108


In [61]:
X_te_test = X_te_test.fillna(0)

In [65]:
for col in X_te_test.columns:
    X_te_test.loc[X_te_test[col].apply(lambda x: type(x) == str), col] = None
    X_te_test[col].fillna(X_te_test[col].mean(), inplace = True)

In [ ]:
# X_te_test.to_csv("X_te_test")

In [68]:
test_te_preds = model_te.predict(X_te_test)

In [75]:
MAE_test_te=mae(test_te_preds,y_test)

In [77]:
MAE_test_te

20861.49616928941

In [76]:
pd.DataFrame({'MAE':{
    'test one hot encoding': MAE_test_ohe,
    'test target encoding': MAE_test_te,
                    }
             })

,MAE
test one hot encoding,27640.063893
test target encoding,20861.496169


In [78]:
from sklearn.linear_model import LinearRegression

In [79]:
model = LinearRegression()

In [80]:
model.fit(Xy_train.drop('SalePrice', axis=1),Xy_train['SalePrice'])

LinearRegression()

In [81]:
r_sq = model.score(Xy_train.drop('SalePrice', axis=1),Xy_train['SalePrice'])
print('coefficient of determination:', r_sq)

coefficient of determination: 0.9149918400435689
